In [ ]:
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 1.6 MB/s eta 0:13:41tcmalloc: large alloc 1147494400 bytes == 0x3995c000 @  0x7f731b372615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████▉                | 1055.7 MB 1.5 MB/s eta 0:11:47tcmalloc: large alloc 1434370048 bytes == 0x7dfb2000 @  0x7f731b372615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████            | 1336.2 MB 1.5 MB/s eta 0:08:41tcmalloc: large alloc 1792966656 bytes =

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 963 kB/s 
     |████████████████████████████████| 74 kB 2.2 MB/s 
     |████████████████████████████████| 147 kB 37.6 MB/s 
     |████████████████████████████████| 130 kB 47.3 MB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 
     |████████████████████████████████| 843 kB 32.6 MB/s 
     |████████████████████████████████| 749 kB 45.7 MB/s 
     |████████████████████████████████| 112 kB 20.4 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220506-py3-none-any.whl size=61284 sha256=2b22e3bb99d1893a44bbf1eebd6bc71d357d00270b59376a19daf13578a341d9
  Stored in directory: /root/.cache/pip/wheels/f5/ef/3c/708de8799f89f0871bd209866831fe3885db93fa090608fa73
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=60eb46df038bcc637ea3ee5228c7ae550

In [ ]:
import glob, os, json
import pandas as pd
data_train = json.load(open("/content/drive/MyDrive/detectron/dataset/rhun_train/train.json"))
data_val = json.load(open("/content/drive/MyDrive/detectron/dataset/rhun_val/val.json"))
df_train = pd.DataFrame(data_train["data"])
df_val = pd.DataFrame(data_val["data"])

In [ ]:
from detectron2.structures import BoxMode
#/content/drive/MyDrive/detectron/dataset/training_images

def get_data_dicts(img_directory, df_train):
    img_dir = img_directory
    dataset_dicts = []
    for i in range(len(df_train)):
        record = {}

        index = df_train["index"][i]
        filename = os.path.join(img_dir, df_train["image"][i])
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = index
        record["height"] = height
        record["width"] = width

        annos = df_train["annotations"]
        objs = []
        for j in range(len(annos[i])):
            x = annos[i][j]["coordinates"]["x"]-(0.5*annos[i][j]["coordinates"]["width"])
            y = annos[i][j]["coordinates"]["y"]-(0.5*annos[i][j]["coordinates"]["height"])
            w = annos[i][j]["coordinates"]["width"]
            h = annos[i][j]["coordinates"]["height"]
            class_num =0 
            if annos[i][j]["label"] == "left":
                class_num = 0
            elif annos[i][j]["label"] == "right":
                class_num = 1
            elif annos[i][j]["label"] == "up":
                class_num = 2
            elif annos[i][j]["label"] == "down":
                class_num = 3
            obj = {
                        "bbox": [x, y, w, h],
                        "bbox_mode": BoxMode.XYWH_ABS,
                        "category_id": class_num#annos[i][j]["label"],
                  }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts



# DatasetCatalog.register("rhun_train", get_data_dicts("/content/drive/MyDrive/detectron/dataset/rhun_train"))
# MetadataCatalog.get("rhun_train").set(thing_classes=["left", "right", "up", "down"])
# rhun_metadata = MetadataCatalog.get("rhun_train")
for d in ["train", "val"]:
    DatasetCatalog.register("rhun_" + d, lambda d=d: get_data_dicts("/content/drive/MyDrive/detectron/dataset/rhun_" + d, df_train))
    MetadataCatalog.get("rhun_" + d).set(thing_classes=["left", "right", "up", "down"])


rhun_metadata = MetadataCatalog.get("rhun_train")

In [ ]:
# Clear datasets
DatasetCatalog.clear()
MetadataCatalog.clear()

In [ ]:
# 
dataset_dicts = get_data_dicts("/content/drive/MyDrive/detectron/dataset/rhun_train", df_train)
dataset_dicts_val = get_data_dicts("/content/drive/MyDrive/detectron/dataset/rhun_val", df_val)


In [ ]:
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=rhun_metadata, scale=1.0)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("rhun_train",)
cfg.DATASETS.TEST = ()
cfg.INPUT.RANDOM_FLIP = "none"
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00030  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[05/06 09:15:16 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_b275ba.pkl: 167MB [00:04, 38.7MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_p

[05/06 09:15:32 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/06 09:16:05 d2.utils.events]:  eta: 0:25:57  iter: 19  total_loss: 2.366  loss_cls: 1.489  loss_box_reg: 0.8383  loss_rpn_cls: 0.02806  loss_rpn_loc: 0.01416  time: 1.5921  data_time: 0.0405  lr: 5.9943e-06  max_mem: 4337M
[05/06 09:16:37 d2.utils.events]:  eta: 0:25:34  iter: 39  total_loss: 2.267  loss_cls: 1.412  loss_box_reg: 0.7874  loss_rpn_cls: 0.02876  loss_rpn_loc: 0.01392  time: 1.5986  data_time: 0.0275  lr: 1.1988e-05  max_mem: 4337M
[05/06 09:17:09 d2.utils.events]:  eta: 0:25:09  iter: 59  total_loss: 2.084  loss_cls: 1.259  loss_box_reg: 0.8013  loss_rpn_cls: 0.02703  loss_rpn_loc: 0.0131  time: 1.6076  data_time: 0.0279  lr: 1.7982e-05  max_mem: 4337M
[05/06 09:17:42 d2.utils.events]:  eta: 0:24:46  iter: 79  total_loss: 1.912  loss_cls: 1.055  loss_box_reg: 0.8113  loss_rpn_cls: 0.02029  loss_rpn_loc: 0.01276  time: 1.6127  data_time: 0.0291  lr: 2.3976e-05  max_mem: 4337M
[05/06 09:18:15 d2.utils.events]:  eta: 0:24:17  iter: 99  total_loss: 1.716  loss_cls: 0.865

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
# Make predictor with trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
# cfg.DATASETS.TEST = ("car_testing_images", )
predictor = DefaultPredictor(cfg)

In [ ]:
# Download config settings
with open("/content/output.yaml", "w") as f: 
  f.write(cfg.dump()) # save config to file


In [ ]:
# testing inference time
import time
from detectron2.utils.visualizer import ColorMode
im = cv2.imread("/content/drive/MyDrive/detectron/dataset/rhun_val/0030.png")
start = time.time()
outputs = predictor(im)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
# testing valdation sets
from detectron2.utils.visualizer import ColorMode
# dataset_dicts_val = get_data_dicts("/content/drive/MyDrive/detectron/dataset/rhun_val", df_val)
for d in random.sample(dataset_dicts_val, 10):    
    im = cv2.imread(d["file_name"])
    start = time.time()
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시
    v = Visualizer(im[:, :, ::-1],
                   metadata=rhun_metadata, 
                   scale=1.0, 
                   instance_mode=ColorMode.IMAGE   
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# empty cache
import gc
gc.collect()
torch.cuda.empty_cache()

WARNING [04/19 12:14:23 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[04/19 12:14:23 d2.evaluation.coco_evaluation]: Trying to convert 'rhun_val' to COCO format ...
[04/19 12:14:23 d2.data.datasets.coco]: Converting annotations of dataset 'rhun_val' to COCO format ...)


AttributeError: ignored

In [ ]:

cfg = get_cfg()
cfg.merge_from_file("/content/output.yaml")
cfg.MODEL.WEIGHTS = ("/content/drive/MyDrive/detectron/model_final.pth")
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # set the testing threshold for this model
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

# cfg.DATASETS.TEST = ("car_testing_images", )
predictor = DefaultPredictor(cfg)